In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import warnings
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
import seaborn as sns

from IPython.display import Image
warnings.filterwarnings('ignore')

In [9]:
train_df = pd.read_csv("dev.csv")
test_df = pd.read_csv("compete.csv")

In [10]:
train_df = train_df.drop(['is_host_login', 'num_outbound_cmds'], axis=1);
test_df = test_df.drop(['is_host_login', 'num_outbound_cmds'], axis=1);

In [11]:
train_df = pd.get_dummies(train_df, columns=['protocol_type'])
test_df = pd.get_dummies(test_df, columns=['protocol_type'])

In [12]:
from sklearn import preprocessing

cat_cols = ['service', 'flag', 'protocol_type']
for col in cat_cols:
    if col in train_df.columns:
        le = preprocessing.LabelEncoder()
        le.fit(list(train_df[col].astype(str).values) + list(test_df[col].astype(str).values))
        train_df[col] = le.transform(list(train_df[col].astype(str).values))
        test_df[col] = le.transform(list(test_df[col].astype(str).values))   

In [13]:
numerical_features = list(train_df.columns[train_df.dtypes != object].values[:-1])
categorical_features = list(train_df.columns[train_df.dtypes == object].values)

corr_table = train_df.corr()
triu = corr_table.where(np.triu(np.ones(corr_table.shape) ,k=1).astype(np.bool))
to_drop = [feat for feat in triu.columns if any(triu[feat] > 0.95)]

train_df = train_df.drop(to_drop, axis=1)

for feat in to_drop:
    if feat in categorical_features:
        categorical_features.remove(feat)
    else:
        numerical_features.remove(feat)

print(f'\nFeatures dropped: {to_drop}')
# plt.figure(figsize=(50, 30))
# _ = sns.heatmap(corr_table, annot=True, fmt='.2f')


Features dropped: ['num_root', 'srv_serror_rate', 'srv_rerror_rate', 'dst_host_same_srv_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'protocol_type_icmp']


In [14]:
from sklearn.model_selection import train_test_split

X = train_df.drop('class', axis=1)
y = train_df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

-------------------HYPERPARAMETERS----------------------------------

In [15]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=100)

In [16]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [17]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=MLPClassifier(max_iter=100), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50),
                                                (100,)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']})

In [18]:
preds = clf.predict(X_test)

acc_mlp = (preds == y_test).sum().astype(float) / len(preds)*100

print("MLP Classifier prediction accuracy is: %3.2f" % (acc_mlp))

MLP Classifier prediction accuracy is: 99.36


In [23]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'solver': 'adam'}
0.992 (+/-0.000) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'sgd'}
0.997 (+/-0.004) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}
0.992 (+/-0.000) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'solver': 'sgd'}
0.996 (+/-0.003) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'solver': 'adam'}
0.992 (+/-0.000) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'constant', 'solver': 'sgd'}
0.995 (+/-0.004) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'constant', 'solver': 'ada

In [25]:
mlp = MLPClassifier(max_iter=100, activation = 'tanh', alpha = 0.0001, hidden_layer_sizes = (100,), learning_rate = 'adaptive', solver = 'adam')
mlp.fit(X_train, y_train)

preds = mlp.predict(X_test)

acc_mlp = (preds == y_test).sum().astype(float) / len(preds)*100

print("MLP Classifier prediction accuracy is: %3.2f" % (acc_mlp))

MLP Classifier prediction accuracy is: 99.64


In [24]:
# from sklearn.neural_network import MLPClassifier

# clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)
# preds = clf.predict(X_test)

# acc_mlp = (preds == y_test).sum().astype(float) / len(preds)*100

# print("MLP Classifier prediction accuracy is: %3.2f" % (acc_mlp))

MLP Classifier prediction accuracy is: 99.71


In [19]:
test_id = test_df.Id.values
test_df = test_df.drop("Id", axis=1)

In [20]:
test_df = test_df.drop(to_drop, axis=1)

In [21]:
preds_MLP = clf.predict(test_df)

In [22]:
submit = pd.DataFrame({'Id': test_id, 'class':preds_MLP})
submit.to_csv('MLP.csv', index=False)